a notebook that trains a GAN; just a generic GANerator.

In [ ]:
# get some image data
!pip install image_datasets
import image_datasets
image_datasets.oslomini.download()

Build a discriminator and generator network; run that puppy.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm

# Hyperparameters
IMG_WIDTH = 64
IMG_HEIGHT = 64
CHANNELS = 3
BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 500
NOISE_DIM = 100

# Data Loading and Preprocessing
def load_images(folder_path):
    """Loads images from a specified folder, resizes them, and normalizes pixel values."""
    images = []
    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(img_path).convert("RGB").resize((IMG_WIDTH, IMG_HEIGHT))
                # Convert to float32 explicitly
                img_array = np.array(img, dtype=np.float32) / 127.5 - 1.0
                images.append(img_array)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(images, dtype=np.float32)  # Ensure final array is float32

def create_dataset(images, buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE):
    """Creates a tf.data.Dataset with proper preprocessing."""
    return (tf.data.Dataset.from_tensor_slices(images.astype('float32'))  # Ensure float32
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .prefetch(tf.data.AUTOTUNE))

@tf.function
def train_step(generator, discriminator, images, generator_optimizer, discriminator_optimizer):
    GRADIENT_PENALTY_WEIGHT = 10.0
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, NOISE_DIM], dtype=tf.float32)  # Specify dtype

    with tf.GradientTape(persistent=True) as tape:
        # Generate images
        generated_images = generator(noise, training=True)

        # Get discriminator outputs
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Add gradient penalty
        alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0, dtype=tf.float32)  # Specify dtype
        interpolated = tf.cast(alpha * images + (1 - alpha) * generated_images, tf.float32)  # Ensure float32

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            interpolated_output = discriminator(interpolated, training=True)

        grads = gp_tape.gradient(interpolated_output, interpolated)
        grad_norms = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gradient_penalty = GRADIENT_PENALTY_WEIGHT * tf.reduce_mean(tf.square(grad_norms - 1.0))

        disc_loss += gradient_penalty

    # Calculate and apply gradients
    gen_gradients = tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Generator
def make_generator_model():
    model = tf.keras.Sequential([
        # Start with a larger initial feature map (8x8)
        tf.keras.layers.Dense(8*8*512, use_bias=False, input_shape=(NOISE_DIM,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),  # Changed from LeakyReLU
        tf.keras.layers.Reshape((8, 8, 512)),

        # Upsample to 16x16
        tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),

        # Upsample to 32x32
        tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),

        # Upsample to 64x64
        tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),

        # Final layer with smaller kernel
        tf.keras.layers.Conv2D(CHANNELS, (3, 3), padding='same', activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = tf.keras.Sequential([
        # 64x64 -> 32x32
        tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                              input_shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS]),
        tf.keras.layers.LayerNormalization(),  # Changed from nothing
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),

        # 32x32 -> 16x16
        tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),

        # 16x16 -> 8x8
        tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),

        # Classification
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, use_bias=False)  # Removed bias to prevent mode collapse
    ])
    return model

# Modified loss functions with label smoothing
def discriminator_loss(real_output, fake_output):
    # Label smoothing: use 0.9 instead of 1.0 for real samples
    # Instantiate BinaryCrossentropy
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = cross_entropy(tf.ones_like(real_output) * 0.9, real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # Instantiate BinaryCrossentropy
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Modified training step with gradient penalty
@tf.function
def train_step(generator, discriminator, images, generator_optimizer, discriminator_optimizer):
    GRADIENT_PENALTY_WEIGHT = 10.0
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, NOISE_DIM])

    with tf.GradientTape(persistent=True) as tape:
        # Generate images
        generated_images = generator(noise, training=True)

        # Get discriminator outputs
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Add gradient penalty
        alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
        interpolated = alpha * images + (1 - alpha) * generated_images

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            interpolated_output = discriminator(interpolated, training=True)

        grads = gp_tape.gradient(interpolated_output, interpolated)
        grad_norms = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gradient_penalty = GRADIENT_PENALTY_WEIGHT * tf.reduce_mean(tf.square(grad_norms - 1.0))

        disc_loss += gradient_penalty

    # Calculate and apply gradients
    gen_gradients = tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

def save_generator(generator, filepath='generator_model'):
    """Save the generator model to the specified path"""
    try:
        tf.keras.models.save_model(generator, filepath) # use standard Keras saving
        print(f"Model successfully saved to {filepath}")
    except Exception as e:
        print(f"Error saving model: {e}")

def train(dataset, epochs):
    # Initialize models and optimizers with lower learning rates
    generator = make_generator_model()
    discriminator = make_discriminator_model()

    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)
    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)

    # Training history
    history = {'gen_loss': [], 'disc_loss': []}

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')

        # Training loop
        for batch in tqdm(dataset):
            gen_loss, disc_loss = train_step(
                generator, discriminator, batch,
                generator_optimizer, discriminator_optimizer
            )

        # Save model every 50 epochs
        if (epoch + 1) % 50 == 0:
            save_generator(generator, f'generator_model_epoch_{epoch + 1}')

        # Print epoch summary
        print(f'Generator Loss: {gen_loss:.4f}')
        print(f'Discriminator Loss: {disc_loss:.4f}')

    # Save final model
    save_generator(generator, 'generator_model_final')
    return generator, discriminator, history

def visualize_and_save(generator, epoch, num_examples=4):
    """Generate and save sample images during training."""
    noise = tf.random.normal([num_examples, NOISE_DIM])
    generated_images = generator(noise, training=False)

    fig = plt.figure(figsize=(4, 4))
    for i in range(num_examples):
        plt.subplot(4, 4, i + 1)
        plt.imshow((generated_images[i] * 0.5 + 0.5))
        plt.axis('off')

    plt.savefig(f'gan_epoch_{epoch}.png')
    plt.close()

# Main execution
if __name__ == "__main__":
    # Load and prepare data
    image_folder = '/content/datasets/oslomini/images/images'
    if not os.path.exists(image_folder):
        raise ValueError(f"Image folder '{image_folder}' not found")

    images = load_images(image_folder)
    if len(images) == 0:
        raise ValueError(f"No valid images found in '{image_folder}'")

    # Create dataset
    dataset = create_dataset(images)

    # Train the model
    generator, discriminator, history = train(dataset, EPOCHS)

    # Final visualization
    visualize_and_save(generator, EPOCHS)

100%|██████████| 1000/1000 [00:05<00:00, 183.73it/s]
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/500


100%|██████████| 3/3 [00:10<00:00,  3.38s/it]


Generator Loss: 0.0774
Discriminator Loss: 26.0311
Epoch 2/500


100%|██████████| 3/3 [00:02<00:00,  1.36it/s]


...

100%|██████████| 3/3 [00:02<00:00,  1.34it/s]


Generator Loss: 1.1814
Discriminator Loss: 1.8058
Epoch 86/500


100%|██████████| 3/3 [00:02<00:00,  1.35it/s]


Generator Loss: 0.6755
Discriminator Loss: 1.8222
Epoch 87/500


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
!zip -r generator.zip generator_model_final/

  adding: generator_model_final/ (stored 0%)
  adding: generator_model_final/saved_model.pb (deflated 88%)
  adding: generator_model_final/assets/ (stored 0%)
  adding: generator_model_final/variables/ (stored 0%)
  adding: generator_model_final/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: generator_model_final/variables/variables.index (deflated 69%)
  adding: generator_model_final/fingerprint.pb (stored 0%)


In [ ]:
!zip -r trainingimages.zip /content/datasets/oslomini/images/images